In [63]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# load ascii text and covert to lowercase
from nltk.corpus import PlaintextCorpusReader
import os
import sys
wordlists = PlaintextCorpusReader("Nepali_Corpus", '.*txt')
data = wordlists.fileids()[:100]
text = []
for i in data:
    with open(os.path.join("Nepali_Corpus",i)) as file:
        text.append(file.read())
text = "\n".join(text)
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])


Total Characters:  166466
Total Vocab:  91


In [64]:
len(dataX[0])

100

In [65]:
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize


Total Patterns:  166366


In [66]:
X[0]/n_vocab

array([[0.59340659],
       [0.32967033],
       [0.63736264],
       [0.69230769],
       [0.53846154],
       [0.54945055],
       [0.72527473],
       [0.61538462],
       [0.01098901],
       [0.94505495],
       [0.01098901],
       [0.56043956],
       [0.7032967 ],
       [0.62637363],
       [0.8021978 ],
       [0.59340659],
       [0.01098901],
       [0.95604396],
       [0.68131868],
       [0.72527473],
       [0.61538462],
       [0.8021978 ],
       [0.61538462],
       [0.75824176],
       [0.96703297],
       [0.62637363],
       [0.75824176],
       [0.01098901],
       [0.61538462],
       [0.7032967 ],
       [0.62637363],
       [0.7032967 ],
       [0.40659341],
       [0.01098901],
       [0.59340659],
       [0.7032967 ],
       [0.49450549],
       [0.7032967 ],
       [0.01098901],
       [0.53846154],
       [0.40659341],
       [0.7032967 ],
       [0.32967033],
       [0.7032967 ],
       [0.28571429],
       [0.67032967],
       [0.17582418],
       [0.351

In [67]:
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)


In [68]:
X.shape

(166366, 100, 1)

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
#model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# load the network weights
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
166366/166366 [==============================] - 730s 4ms/step - loss: 3.4115

Epoch 00001: loss improved from inf to 3.41154, saving model to weights-improvement-01-3.4115.hdf5
Epoch 2/20
166366/166366 [==============================] - 692s 4ms/step - loss: 3.2199

Epoch 00002: loss improved from 3.41154 to 3.21993, saving model to weights-improvement-02-3.2199.hdf5
Epoch 3/20
166366/166366 [==============================] - 688s 4ms/step - loss: 3.1118

Epoch 00003: loss improved from 3.21993 to 3.11184, saving model to weights-improvement-03-3.1118.hdf5
Epoch 4/20
166366/166366 [==============================] - 733s 4ms/step - loss: 3.0102

Epoch 00004: loss improved from 3.11184 to 3.01019, saving model to weights-improvement-04-3.0102.hdf5
Epoch 5/20
166366/166366 [==============================] - 681s 4ms/step - loss: 2.9220

Epoch 00005: loss improved from 3.01019 to 2.92203, saving model to weights-improvement-05-2.9220.hdf5
Epoch 6/20
166366/166366 [=============

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
gen_data = [int_to_char[value] for value in pattern]
gen_data.append("***")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    print(prediction)
    result = int_to_char[index]
    try:
        gen_data.append(int_to_char[result])
    except:
        gen_data.append(" ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

In [ ]:
"".join(gen_data)

In [ ]:
print(int_to_char)